In [1]:

import pandas as pd
import re
import ast
import numpy as np
import pandas as pd
import glob
import json

In [8]:
"""
Input

metadata_csv_path - Path to latest metadata.csv file from Kaggle's CORD-19 dataset.
covid_19_term_list_path - Path to text file containing a list of Covid-19 synonyms (1 per line).

"""
resources_path = 'data/'
metadata_csv_path = 'data/metadata.csv'
covid_19_term_list_path = 'data/covid_19_terms.txt'

pub_date_cutoff = '2019-10-01'

pq_text_file_dir = '%sv8_data_P/' % resources_path

#Columns to truncate pq full text files to
pq_text_files_cols_oi = ['cord_uid', 'sentence_id', 'section', 'sentence']

#drug path
drug_terms_path='%sDrugNames.txt' % resources_path

ade_terms_path='%sCTCAE_v5.0.xlsx' % resources_path
"""
Output

Filtered metadata dataframe is saved to csv file.

"""

output_path = 'data/output/'

covid19_filtered_uid_list_file = '%scovid19_cord_uids_200429.txt' % output_path

csv_trunc_text_file_dir = '%sv8_truncated_text_files/' % output_path

drug_mentioned_file='%sdrug_mentions.csv' % output_path



In [9]:
drug_mentioned_file

'data/output/drug_mentions.csv'

In [3]:
def filter_metadata_df_by_title_abstract_terms(metadata_df, covid_19_term_list_path):
    """
    Filter metadata dataframe to publications containing a Covid-19 synonym in title or abstract.
    """
        
    #Concatenate title and abstract text into a single, lower-cased column
    
    metadata_df = metadata_df.fillna('')
    metadata_df.loc[:, 'title_abstract'] = metadata_df.loc[:, 'title'].str.lower() + ' ' + metadata_df.loc[:, 'abstract'].str.lower()
    metadata_df.loc[:, 'title_abstract'] = metadata_df.loc[:, 'title_abstract'].fillna('')

    #Load text file containing Covid-19 synonyms (1 per line) and generate a search pattern
    with open(covid_19_term_list_path) as f:
        covid_19_terms = f.read().splitlines()
        covid_19_term_pattern = '|'.join([i.lower() for i in covid_19_terms])

    covid19_df = metadata_df.loc[metadata_df.title_abstract.str.contains(covid_19_term_pattern)]
    
    return covid19_df

def filter_metadata_df_by_publ_date_cutoff(metadata_df, pub_date_cutoff):
    
    date_filtered_df = metadata_df.loc[metadata_df['publish_time'] > pub_date_cutoff]

    return date_filtered_df




In [4]:
#Main

#Load metadata.csv as dataframe and filter to 
metadata_df = pd.read_csv(metadata_csv_path)
covid19_df = filter_metadata_df_by_title_abstract_terms(metadata_df, covid_19_term_list_path)

#Filter covid19_df by publication date cutoff
covid19_date_filtered_df = filter_metadata_df_by_publ_date_cutoff(covid19_df, pub_date_cutoff)

covid19_date_filtered_df.to_csv('data/covid19_date_filt_metadata_200504.csv')

/home/aradhana/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
covid19_date_filtered_df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,title_abstract
11763,ene62kcz,26f8c5fbd310c95c2e1fb04c34ed6f5d10901d07,PMC,Evaluating the potential impact of targeted va...,10.1186/s12976-019-0112-6,PMC6778978,3.15877e+07,cc-by,BACKGROUND: Severe Acute Respiratory Syndrome ...,2019-10-07,"Abdirizak, Fatima; Lewis, Rayleen; Chowell, Ge...",Theor Biol Med Model,,,,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,evaluating the potential impact of targeted va...
11766,uw41ygmc,cf12680358351109a975a23165bbb48620b578a5,PMC,Development of a recombinant replication-defic...,10.1371/journal.pone.0223684,PMC6779238,3.15897e+07,cc-by,Middle East respiratory syndrome-coronavirus (...,2019-10-07,"Kato, Hirofumi; Takayama-Ito, Mutsuyo; Iizuka-...",PLoS One,,,,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,development of a recombinant replication-defic...
11851,4vgnzvag,8bb39b846b0bd9548526bad9e74fed1e4a8ac4cb,PMC,Molecular epidemiology of respiratory viruses ...,10.1186/s12917-019-2103-6,PMC6802313,3.1639e+07,cc-by,BACKGROUND: Viral diseases are a matter of gre...,2019-10-21,"Umar, Sajid; Teillaud, Angélique; Aslam, Hassa...",BMC Vet Res,,,,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,molecular epidemiology of respiratory viruses ...
11861,7unnfmks,058f046fec5eee31c9ed0bdacd37aa22ba485993,PMC,Clinical outcomes among hospital patients with...,10.1186/s12879-019-4555-5,PMC6805532,3.16406e+07,cc-by,BACKGROUND: Mortality is high among patients w...,2019-10-22,"Habib, Abdulrahman Mohammed G.; Ali, Mohamed A...",BMC Infect Dis,,,,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,clinical outcomes among hospital patients with...
11873,8opynjru,a0839bf8f94d79c1253bf89f170c22f9342fc095,PMC,"Middle East Respiratory Syndrome Coronavirus, ...",10.3201/eid2511.190726,PMC6810214,3.14302e+07,no-cc,We characterized exposures and demographics of...,2019-11-01,"Hakawi, Ahmed; Rose, Erica Billig; Biggs, Holl...",Emerg Infect Dis,,,,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,"middle east respiratory syndrome coronavirus, ..."


In [6]:
def truncate_pq_text_file(pq_text_file, pq_text_files_cols_oi, uid_list):
    """
    Load text file as dataframe, filter to relevant columns (cord_uid, sentence_id, section, sentence) and 
    cord_uids for Covid-19 papers.    
    
    Return truncated dataframe.
    
    """
    print("Loading dataframe from %s" % pq_text_file)
    text_file_df = pd.read_parquet(pq_text_file)
    text_file_trunc_df = text_file_df[pq_text_files_cols_oi]
    text_file_trunc_df = text_file_trunc_df.loc[text_file_trunc_df.cord_uid.isin(uid_list)]
    
    return text_file_trunc_df
    
def truncate_pq_text_files(pq_text_file_dir, pq_text_files_cols_oi, csv_trunc_text_file_dir, uid_list):
    """
    For each full text parquet file in pq_text_file_dir, load as dataframe, 
    filter to specified cord_uids and columns of interest, and write to csv file in csv_trunc_text_file_dir
    """
    
    print("Parquet text files input directory: %s" % pq_text_file_dir)
    pq_text_files = glob.glob('%s*' % pq_text_file_dir)
    print("Files in text files input directory: %d" % len(pq_text_files))

    print("Parquet text files output directory: %s" % csv_trunc_text_file_dir)
    print('\n')

    for pq_text_file in pq_text_files:
        
        text_file_trunc_df = truncate_pq_text_file(pq_text_file, pq_text_files_cols_oi, uid_list)
        text_file_name = pq_text_file.split('.')[0].split('/')[-1]
        outfile = '%s%s_trunc.csv' %(csv_trunc_text_file_dir, text_file_name)
    
        print("Writing truncated text dataframe to %s" % outfile)
        text_file_trunc_df.to_csv(outfile)

In [7]:
def concat_dataframes_from_dir_csvs(trunc_csv_dir):

    trunc_csv_files = glob.glob('%s*' % trunc_csv_dir)
    print("Files in truncated csv output directory: %d" % len(trunc_csv_files))

    trunc_dfs = []
    for trunc_csv_file in trunc_csv_files:
        trunc_dfs.append(pd.read_csv(trunc_csv_file, index_col=0))
    
    merged_trunc_df = pd.concat(trunc_dfs)
    return merged_trunc_df

In [8]:

#Main

print("Step 1: Filtering metadata csv by Covid-19 synoynms and publication date...")

#covid19_df = pd.read_csv(metadata_csv_path)
uid_list = covid19_date_filtered_df.cord_uid.tolist()

print("Step 2: Loading full text dataframes, filtering to extracted uids, and writing to csv files...")
truncate_pq_text_files(pq_text_file_dir, pq_text_files_cols_oi, csv_trunc_text_file_dir, uid_list)    
print('\n')

Step 1: Filtering metadata csv by Covid-19 synoynms and publication date...
Step 2: Loading full text dataframes, filtering to extracted uids, and writing to csv files...
Parquet text files input directory: data/v8_data_P/
Files in text files input directory: 20
Parquet text files output directory: data/output/v8_truncated_text_files/


Loading dataframe from data/v8_data_P/v8processedText3.parquet
Writing truncated text dataframe to data/output/v8_truncated_text_files/v8processedText3_trunc.csv
Loading dataframe from data/v8_data_P/v8processedText10.parquet
Writing truncated text dataframe to data/output/v8_truncated_text_files/v8processedText10_trunc.csv
Loading dataframe from data/v8_data_P/v8processedText18.parquet
Writing truncated text dataframe to data/output/v8_truncated_text_files/v8processedText18_trunc.csv
Loading dataframe from data/v8_data_P/v8processedText4.parquet
Writing truncated text dataframe to data/output/v8_truncated_text_files/v8processedText4_trunc.csv
Loading d

In [9]:
print("Step 3: Merging truncated text dataframes and extracting sentence ids...")
concat_text_trunc_df = concat_dataframes_from_dir_csvs(csv_trunc_text_file_dir)
text_sentence_ids = concat_text_trunc_df.sentence_id.tolist()
print("Number of sentence ids: %d" % len(text_sentence_ids))
print("Number of unique sentence ids: %d" % len(set(text_sentence_ids)))
print('\n')

Step 3: Merging truncated text dataframes and extracting sentence ids...
Files in truncated csv output directory: 20
Number of sentence ids: 336314
Number of unique sentence ids: 336314




In [14]:
"""
Read drug terms from reference file and generate regex pattern for matching.
"""

with open(drug_terms_path) as f:
    drug_terms = ['%s' % i.lower() for i in f.read().splitlines()]
    
drug_terms_pattern =  '\W' + '\W|\W'.join(drug_terms) + '\W'

In [40]:
"""
Filter to rows where section text contains drug term regex pattern.
"""
print(concat_text_trunc_df[concat_text_trunc_df.isnull().any(axis=1)])
concat_text_trunc_df=concat_text_trunc_df.dropna()
contain_drug_mask = concat_text_trunc_df['sentence'].str.contains(drug_terms_pattern, case=False)
#section_text_with_drugs_df = concat_text_trunc_df[contain_drug_mask]

Empty DataFrame
Columns: [cord_uid, sentence_id, section, sentence]
Index: []


/home/aradhana/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [43]:
section_text_with_drugs_df = concat_text_trunc_df[contain_drug_mask]

In [29]:
#covid19_date_filtered_df['cord_uid'=='m201s56m']
covid19_date_filtered_df[covid19_date_filtered_df['cord_uid'].str.contains('m201s56m')]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,title_abstract
45394,m201s56m,4ecc308121e248cad3f81a899c0e2df3a4484421; 622d...,Elsevier,A UK-wide British Society of Thoracic Imaging ...,10.1016/j.crad.2020.03.005,PMC7138155,,els-covid,,2020-05-31,"Hare, S.S.; Rodrigues, J.C.L.; Jacob, J.; Edey...",Clinical Radiology,,,,True,False,custom_license,https://doi.org/10.1016/j.crad.2020.03.005,a uk-wide british society of thoracic imaging ...


In [45]:
section_text_with_drugs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12710 entries, 16996 to 760
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cord_uid     12710 non-null  object
 1   sentence_id  12710 non-null  object
 2   section      12710 non-null  object
 3   sentence     12710 non-null  object
dtypes: object(4)
memory usage: 496.5+ KB


In [46]:
section_text_with_drugs_df.to_csv("data/output/drug_mentions.csv")

In [16]:
# """
# Manual annotation of drug terms used.

# """
#read Drug mentioned file
section_text_with_drugs_df=pd.read_csv(drug_mentioned_file)
section_text_with_drugs_ann_df = section_text_with_drugs_df.copy()

section_text_with_drugs_ann_df['drug_terms_used'] = ''
section_text_with_drugs_ann_df['drug_count'] = 0
for index, row in section_text_with_drugs_ann_df.iterrows():

    
    drugs_used = []
    count=0
    for drug in drug_terms:
        if drug in row.sentence.lower():
            drugs_used.append(drug)
            count+=1
    section_text_with_drugs_ann_df.at[index, 'drug_terms_used'] = ','.join(drugs_used)
    section_text_with_drugs_ann_df.at[index, 'drug_count'] = count
section_text_with_drugs_ann_df.to_csv('data/output/section_texts_with_drug_mentions_ann_200516.csv')

In [18]:
section_text_with_drugs_ann_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12710 entries, 0 to 12709
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       12710 non-null  int64 
 1   cord_uid         12710 non-null  object
 2   sentence_id      12710 non-null  object
 3   section          12710 non-null  object
 4   sentence         12710 non-null  object
 5   drug_terms_used  12710 non-null  object
 6   drug_count       12710 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 695.2+ KB


In [19]:
"Read ADE file and prepare list"
ade=pd.read_excel(ade_terms_path)
ade['CTCAE Term']=ade['CTCAE Term'].apply(lambda x:x.lower())
ade_list=ade['CTCAE Term'].values.tolist()


In [21]:
#Now,Its time to find adverse drug event mentioned in small subset which contains drug

#section_text_with_drugs_ann_df = concat_text_trunc_df.copy()

section_text_with_drugs_ann_df['ade_terms_used'] = ''
section_text_with_drugs_ann_df['ade_count'] = 0
for index, row in section_text_with_drugs_ann_df.iterrows():
    
    ade_used = []
    count=0
    for ad_event in ade_list:
        if ad_event in row.sentence.lower():
            ade_used.append(ad_event)
            count+=1
    section_text_with_drugs_ann_df.at[index, 'ade_terms_used'] = ','.join(ade_used)
    section_text_with_drugs_ann_df.at[index, 'ade_count'] = count

In [22]:
section_text_with_drugs_ann_df.head()

,Unnamed: 0,cord_uid,sentence_id,section,sentence,drug_terms_used,drug_count,ade_terms_used,ade_count
0,16996,xhaltyff,xhaltyff1816996,Concept 2-obstacles and opportunities in pre-c...,The point was reiterated that we need to stren...,align,1,,0
1,27196,kaip0d1i,kaip0d1i627196,Support vector machine,SVM with a radial kernel is used to perform th...,perform,1,,0
2,34241,8xx1xm2d,8xx1xm2d034241,body_text,"In this Journal, Zhu et al. recently reported ...","colistin,ella,ting",3,,0
3,34243,8xx1xm2d,8xx1xm2d034243,body_text,2 Oral non-absorbed colistin has been proposed...,colistin,1,,0
4,34247,8xx1xm2d,8xx1xm2d034247,body_text,"Here, we studied the impact of non-absorbed or...",colistin,1,,0


In [23]:
section_text_with_drugs_ann_df.to_csv("data/output/drug_ade_dataset_covid.csv")

In [24]:
"""filter out sentences where one drug ,one ADE event is mentioned
"""
one_drug_ade_df=section_text_with_drugs_ann_df[(section_text_with_drugs_ann_df['drug_count']==1) & (section_text_with_drugs_ann_df['ade_count']==1)]

In [25]:
one_drug_ade_df


,Unnamed: 0,cord_uid,sentence_id,section,sentence,drug_terms_used,drug_count,ade_terms_used,ade_count
41,24537,hl6mqd9j,hl6mqd9j024537,body_text,Hypoxia as an often seen problem is frequently...,glucose,1,hypoxia,1
42,24538,hl6mqd9j,hl6mqd9j024538,body_text,"Indeed, a large proportion of the COVID-19 pat...",lactate,1,hyperglycemia,1
156,18861,k36rymkv,k36rymkv2018861,Results,"In non-survivors, high-sensitivity cardiac tro...",lactate,1,cardiac troponin i increased,1
357,28726,vxyrsy83,vxyrsy836828726,Prevention ::: Dengue Fever ::: Specific Infec...,Eliminate any standing water in and around the...,water,1,depression,1
362,17095,k8m2xz0e,k8m2xz0e2117095,SARS and MERS Illness and Complications ::: Cl...,"Laboratory abnormalities include lymphopenia, ...",creatinine,1,acute kidney injury,1
...,...,...,...,...,...,...,...,...,...
12178,27532,p91hfogv,p91hfogv027532,title,New therapeutic opportunities for COVID-19 pat...,tocilizumab,1,osteonecrosis,1
12203,27790,q4u5t8zs,q4u5t8zs3027790,COVID-19-Specific Preoperative Planning and Pr...,"For example, anesthesia may consider avoiding ...",lidocaine,1,cough,1
12225,5997,769aw2iv,769aw2iv05997,abstract,(II) To date we have no data regarding how man...,date,1,conjunctivitis,1
12457,14065,kn1lcsia,kn1lcsia1214065,Long QT syndrome and COVID-19 126,"In addition, physicians should be aware of 159...",chloroquine,1,hypotension,1


In [26]:
"""filter out sentences where more than one drug ,more than one ADE event is mentioned
"""
gt_one_drug_ade_df=section_text_with_drugs_ann_df[(section_text_with_drugs_ann_df['drug_count']>1) & (section_text_with_drugs_ann_df['ade_count']>1)]

In [27]:
gt_one_drug_ade_df

,Unnamed: 0,cord_uid,sentence_id,section,sentence,drug_terms_used,drug_count,ade_terms_used,ade_count
40,24536,hl6mqd9j,hl6mqd9j024536,body_text,"Typically, pulmonary inflammation and compromi...","air,compro,nf",3,"dyspnea,hypoxia",2
115,26676,nxfr24sp,nxfr24sp626676,CASE REPORT,The potential side effects of lopinavir/ritona...,"lopinavir,ritonavir",2,"diarrhea,nausea,dizziness",3
130,23023,wyz5jyjh,wyz5jyjh3423023,Intelligent assisted treatment using nCapp,We should pay attention to the adverse reactio...,"lopinavir,react,ritonavir,ting",4,"diarrhea,nausea,vomiting",3
351,28679,vxyrsy83,vxyrsy8318128679,Illness ::: Viral Haemorrhagic Fevers ::: Spec...,"Ebola is also often associated with headache, ...","injection,ting",2,"vomiting,headache",2
365,17144,k8m2xz0e,k8m2xz0e1317144,Measles Transmission and Mechanisms of Disease...,Measles is among the most highlycontagious res...,"air,nf",2,"cough,sneezing",2
...,...,...,...,...,...,...,...,...,...
12302,10103,b518n9dx,b518n9dx1910103,Feasibility of trials on remdesivir on COVID-19,The focal injection site reactions only showed...,"injection,react,ting",3,"injection site reaction,pain",2
12320,10653,f13iyvh6,f13iyvh62810653,Discussion,"Chinese inpatients, of whom 95% received antib...","lopinavir,ritonavir",2,"fever,cough",2
12422,13296,z4h1ahha,z4h1ahha1013296,Lobe Distribution and Number of Lobes Involved,There was a low probability of publication bia...,"air,ibu,icar",3,"pericardial effusion,pleural effusion",2
12425,13337,z4h1ahha,z4h1ahha413337,Data Extraction,The recorded chest CT imaging features mainly ...,"air,ibu,icar",3,"pericardial effusion,pleural effusion",2


In [28]:
"""filter out sentences where more than one drug ,more than one ADE event is mentioned
"""
gt_one_drug_one_ade_df=section_text_with_drugs_ann_df[(section_text_with_drugs_ann_df['drug_count']>1) & (section_text_with_drugs_ann_df['ade_count']>=1)]
gt_one_drug_one_ade_df

,Unnamed: 0,cord_uid,sentence_id,section,sentence,drug_terms_used,drug_count,ade_terms_used,ade_count
11,34439,8xx1xm2d,8xx1xm2d7734439,Novel coronavirus disease (Covid-19): The firs...,Repeat blood tests in this individual on d3 de...,"creatinine,pea",2,acute kidney injury,1
13,34541,8xx1xm2d,8xx1xm2d4834541,"Dear Editor,",We recently observed the reactivation of Leish...,"methotrexate,nf,react",3,mania,1
40,24536,hl6mqd9j,hl6mqd9j024536,body_text,"Typically, pulmonary inflammation and compromi...","air,compro,nf",3,"dyspnea,hypoxia",2
56,25907,rq9vij1y,rq9vij1y2025907,Discussion,"However, a study conducted Spain found similar...","air,beef",2,pain,1
88,15298,40bshqly,40bshqly115298,Introduction,Polyprotein 1a (pp1a) and polyprotein 1 ab (pp...,"depen,papain,sps",3,pain,1
...,...,...,...,...,...,...,...,...,...
12524,15693,2wnsfkls,2wnsfkls315693,Declaration of conflicting interest,"The hemogasanalysis showed hypoxia, laboratory...","lactate,react",2,hypoxia,1
12526,15698,2wnsfkls,2wnsfkls415698,Declaration of conflicting interest,"He was sedated for agitation and, for this rea...","chloroquine,date,hydroxychloroquine,lopinavir",4,agitation,1
12530,19229,9vrny92m,9vrny92m2019229,IL-6 and IL-1 blockers,Data from a phase 3 RCT of IL-1 blockade (anak...,"anakinra,nf",2,sepsis,1
12540,19265,9vrny92m,9vrny92m1019265,Corticosteroids and NSAIDs,"In addition, the use of both NSAIDs and acetam...","acetaminophen,nf,ting",3,fever,1


In [29]:
"""filter out sentences where  one drug ,more than one ADE event is mentioned
"""
one_drug_gt_ade_df=section_text_with_drugs_ann_df[(section_text_with_drugs_ann_df['drug_count']==1) & (section_text_with_drugs_ann_df['ade_count']>1)]
one_drug_gt_ade_df

,Unnamed: 0,cord_uid,sentence_id,section,sentence,drug_terms_used,drug_count,ade_terms_used,ade_count
14,34545,8xx1xm2d,8xx1xm2d5034545,"Dear Editor,",Considering the above findings leishmaniasis w...,perform,1,"mania,aspiration",2
77,11603,j3avpu1y,j3avpu1y1411603,Results,She developed fever and dry cough and attended...,cefazolin,1,"fever,cough",2
110,21342,4vrqzbof,4vrqzbof2121342,Discussion,"Moreover, given the novelty of the COVID-19 pn...",date,1,"fever,dyspnea",2
113,26662,nxfr24sp,nxfr24sp326662,CASE REPORT,The patient was admitted to the hospital 2 wee...,air,1,"fever,myalgia",2
143,10971,5fg87lvu,5fg87lvu610971,Symptoms,Symptom of CoVID-19 are non-specific and the d...,date,1,"diarrhea,fatigue,fever,myalgia,headache,cough",6
198,6149,gsaw1snc,gsaw1snc56149,Support to staff who are providing end-of-life...,Being responsible for decisions around resourc...,perform,1,"fatigue,burn",2
881,27057,v13k3mm2,v13k3mm22427057,Precision prevention and control for 2019-nCoV...,period.16 The National Health Commission also ...,water,1,"cough,sneezing",2
1278,3253,ud6tuchk,ud6tuchk13253,CASE PRESENTATION,She was found to have acute kidney injury with...,creatinine,1,"acute kidney injury,chronic kidney disease",2
1428,25829,ykofrn9i,ykofrn9i2325829,Figure 2. Estimated cumulative distribution fo...,"Moreover, given the novelty of the COVID-19 pn...",date,1,"fever,dyspnea",2
1785,29398,82jdp391,82jdp391329398,Case report,The patient's fever improved with Tylenol and ...,tylenol,1,"fever,cough,dyspnea",3
